In [1]:
# Import Required Modules
from flask import Flask, render_template
import pandas as pd
import json
import plotly
import plotly.express as px
import pandas_ta as pta
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import date
from datetime import datetime
from datetime import timedelta, date
from prophet import Prophet
import pandas as pd
import numpy as np
import yfinance as yf
import seaborn as sns
import mpld3
import matplotlib.pyplot as plt
import warnings

   
df = pd.read_csv("Resources/btcjoin.csv", parse_dates=['date'])
btc_df = yf.Ticker('BTC-USD').history(period='7y',interval='1d',actions=False).reset_index()
btc_df = btc_df.loc[(btc_df['Date'] > '2022-10-25')]
btc_df['Close']=btc_df['Close'].astype("float")
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
btc_df = btc_df.rename(columns={"Close": "price", "Date":"date"})
btc_df['date'] = btc_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
btc_df['date'] = pd.to_datetime(btc_df['date'])

df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = df.rename(columns={"value": "wallets"})
df = df.drop(columns=['volume','change', 'low', 'high', 'open','Unnamed: 0', "wallets", "address", "mined","Open_x","High_x","Low_x","Volume_x","Open_y","High_y","Low_y","Volume_y"])
df

c:\Users\chris\anaconda3\envs\python_data\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


,date,price
0,2010-09-25,0.100000
1,2010-09-26,0.100000
2,2010-09-27,0.100000
3,2010-09-28,0.100000
4,2010-09-29,0.100000
...,...,...
5176,2024-12-24,98676.093750
5177,2024-12-25,99299.195312
5178,2024-12-26,95795.515625
5179,2024-12-27,94164.859375


In [2]:
mstr_df = yf.Ticker('MSTR').history(period='7y',interval='1d',actions=False).reset_index()
mstr_df = mstr_df.rename(columns={"Date": "date"})
mstr_df['date'] = mstr_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
mstr_df['price'] = mstr_df[['Open', 'High', 'Low', 'Close']].mean(axis=1)
mstr_df = mstr_df.drop(columns=['Volume', 'Open', 'High', 'Low', 'Close'])
# Convert 'date' columns to datetime
df['date'] = pd.to_datetime(df['date'])
mstr_df['date'] = pd.to_datetime(mstr_df['date'])


# Merge the dataframes on the 'date' column
combined_df = pd.merge(df, mstr_df, on='date', how='outer')
combined_df = combined_df.rename(columns={"price_x": "bitcoin", "price_y": "mstr"})
# Drop rows with any NaN values
combined_df = combined_df.dropna()
# Filter the combined dataframe to only include data after August 2020
combined_df = combined_df[combined_df['date'] > '2020-08-01']
# Create a new column 'mstr_to_bitcoin' by dividing 'mstr' by 'bitcoin'
combined_df['mstr_to_bitcoin'] = combined_df['mstr']*1000 / combined_df['bitcoin']
# Display the combined dataframe
# Calculate moving averages
combined_df['mstr_to_bitcoin_7d'] = combined_df['mstr_to_bitcoin'].rolling(window=7).mean()
combined_df['mstr_to_bitcoin_20d'] = combined_df['mstr_to_bitcoin'].rolling(window=20).mean()
combined_df['mstr_to_bitcoin_50d'] = combined_df['mstr_to_bitcoin'].rolling(window=50).mean()
combined_df['mstr_to_bitcoin_100d'] = combined_df['mstr_to_bitcoin'].rolling(window=100).mean()
# Keep only the specified columns
combined_df = combined_df[['date', 'bitcoin', 'mstr','mstr_to_bitcoin', 'mstr_to_bitcoin_20d']]
combined_df['mstr_to_bitcoin_diff'] = combined_df['mstr_to_bitcoin'] - combined_df['mstr_to_bitcoin_20d']
# Calculate the standard deviation of 'mstr_to_bitcoin_diff'
std_dev = combined_df['mstr_to_bitcoin_diff'].std()

# Define the bins based on standard deviation
bins = [-3*std_dev, -std_dev, 0, std_dev, 3*std_dev]

# Create labels for the bins
group_names = ['-3 STD', '-1 STD', '+1 STD', '+3 STD']
combined_df = combined_df.dropna()
# Create a new column 'std_dev_bucket' with the bin labels
combined_df['std_dev_bucket'] = pd.cut(combined_df['mstr_to_bitcoin_diff'], bins=bins, labels=group_names)
fig = px.scatter(combined_df, x="date", y="mstr_to_bitcoin", color="std_dev_bucket", 
                 color_discrete_sequence=["blue", "green", "orange", "red"],
                 title="MSTR to Bitcoin Ratio with Standard Deviation Buckets")
fig.add_scatter(x=combined_df['date'], y=combined_df['mstr_to_bitcoin_20d'], mode='lines', name='20 Day MA')
fig.update_traces(marker=dict(color='black'), selector=dict(name='20 Day MA'))
fig.update_layout(width=1500, height=700)


# Add a horizontal gauge bar
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=combined_df['mstr_to_bitcoin_diff'].iloc[-1],
    domain={'x': [0.1, 0.9], 'y': [0.85, 1]},
    title={'text': "MSTR to Bitcoin Diff"},
    gauge={
        'shape': "bullet",
        'axis': {'range': [-3*std_dev, 3*std_dev]},
        'bar': {'color': "black"},
        'steps': [
            {'range': [-3*std_dev, -1.5*std_dev], 'color': "red"},
            {'range': [-1.5*std_dev, 0], 'color': "orange"},
            {'range': [0, 1.5*std_dev], 'color': "blue"},
            {'range': [1.5*std_dev, 3*std_dev], 'color': "green"}
        ],
        'threshold': {
            'line': {'color': "black", 'width': 4},
            'thickness': 0.3,
            'value': combined_df['mstr_to_bitcoin_diff'].iloc[-1]
        }
    }
))
# Calculate the upper and lower Bollinger Bands
combined_df['upper_band'] = combined_df['mstr_to_bitcoin_20d'] + (combined_df['mstr_to_bitcoin_diff'].std() * 2)
combined_df['lower_band'] = combined_df['mstr_to_bitcoin_20d'] - (combined_df['mstr_to_bitcoin_diff'].std() * 2)



# Add filled areas for the Bollinger Bands
fig.add_trace(go.Scatter(
    x=combined_df['date'],
    y=combined_df['upper_band'],
    mode='lines',
    name='Upper Bollinger Band',
    line=dict(color='rgba(173, 216, 230, 0.9)'),
    fill=None
))

fig.add_trace(go.Scatter(
    x=combined_df['date'],
    y=combined_df['lower_band'],
    mode='lines',
    name='Lower Bollinger Band',
    line=dict(color='rgba(173, 216, 230, 0.9)'),
    fill='tonexty'
))


fig.update_traces(
    domain={'x': [0.4, 0.9], 'y': [0.85, 0.95]},
    selector=dict(type='indicator')
)
fig.show()

c:\Users\chris\anaconda3\envs\python_data\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
